In [169]:
import pandas as pd

In [170]:
poi_df = pd.read_csv('Data/Master Sets/NYC Master/NYC_Subway_POI_Assignment.csv')
frequency = 'monthly'

In [171]:
poi_df.head()

,Unnamed: 0,the_geom,SEGMENTID,COMPLEXID,SAFTYPE,SOS,PLACEID,FACI_DOM,BIN,BOROUGH,...,PRI_ADD,NAME,latitude,longitude,closest_station_id,closest_complex_id,closest_gtfs_id,closest_station_name,closest_line_name,distance_to_station
0,0,POINT (-74.00701717096757 40.724634757833414),31895,0,N,1.0,567,9,0,1.0,...,0,HOLLAND,40.724635,-74.007017,325,325,135,Canal St,Broadway - 7Av,0.001928
1,1,POINT (-73.82661642130311 40.797182526598505),306303,3378,N,2.0,568,8,0,4.0,...,0,WHITESTONE,40.797183,-73.826616,447,447,701,Flushing-Main St,Flushing,0.037737
2,2,POINT (-73.99395441100663 40.70384707235758),144842,3960,N,2.0,576,8,0,3.0,...,0,BROOKLYN,40.703847,-73.993954,173,173,A40,High St,8th Av - Fulton St,0.005662
3,3,POINT (-73.9919414213091 40.70960010711745),162664,0,N,1.0,580,8,0,1.0,...,0,MANHATTAN,40.709600,-73.991941,234,234,F16,East Broadway,6th Av - Culver,0.004479
4,4,POINT (-73.9526609766105 40.73906602249743),157362,0,N,1.0,582,8,0,3.0,...,0,PULASKI,40.739066,-73.952661,464,464,721,Vernon Blvd-Jackson Av,Flushing,0.003677


In [172]:
poi_df['CREATED'] = pd.to_datetime(poi_df['CREATED']).dt.to_period('M').dt.to_timestamp()  # Ensure datetime format

C:\Users\shygu\AppData\Local\Temp\ipykernel_10792\4169500957.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  poi_df['CREATED'] = pd.to_datetime(poi_df['CREATED']).dt.to_period('M').dt.to_timestamp()  # Ensure datetime format


In [173]:
poi_df['CREATED']

0       2009-05-01
1       2009-05-01
2       2009-05-01
3       2009-05-01
4       2009-05-01
           ...    
20591   2024-12-01
20592   2024-12-01
20593   2024-12-01
20594   2025-01-01
20595   2025-01-01
Name: CREATED, Length: 20596, dtype: datetime64[ns]

In [174]:
poi_df.dtypes

Unnamed: 0                       int64
the_geom                        object
SEGMENTID                        int64
COMPLEXID                        int64
SAFTYPE                         object
SOS                            float64
PLACEID                          int64
FACI_DOM                         int64
BIN                              int64
BOROUGH                        float64
CREATED                 datetime64[ns]
MODIFIED                        object
FACILITY_T                       int64
SOURCE                          object
B7SC                           float64
PRI_ADD                          int64
NAME                            object
latitude                       float64
longitude                      float64
closest_station_id               int64
closest_complex_id               int64
closest_gtfs_id                 object
closest_station_name            object
closest_line_name               object
distance_to_station            float64
dtype: object

In [175]:
def generate_poi_ts(date_index, node_index, frequency):

    if frequency == 'monthly':
        period = 'M'
        freq = 'MS'
    elif frequency == 'daily':
        period = 'D'
    
    poi_df[date_index] = pd.to_datetime(poi_df[date_index]).dt.to_period(period).dt.to_timestamp()  # Ensure datetime format

    # Count POIs per day per complex
    poi_counts = poi_df.groupby([date_index, node_index]).size().reset_index(name='poi_count')

    # Pivot the table to get a time series format
    poi_timeseries = poi_counts.pivot(index=date_index, columns=node_index, values='poi_count')

    # Fill missing values with 0 (some complexes might not have POIs on certain days)
    poi_timeseries = poi_timeseries.fillna(0)

    # Ensure all days are present by reindexing with a full date range
    poi_timeseries = poi_timeseries.reindex(pd.date_range(start=poi_df[date_index].min(), 
                                                        end=poi_df[date_index].max(), 
                                                        freq=freq), 
                                            fill_value=0)

    # Convert to cumulative sum to track POIs created over time
    poi_timeseries = poi_timeseries.cumsum()

    # Rename index
    poi_timeseries.index.name = 'date'

    # Reset column names (optional)
    poi_timeseries.columns.name = None

    return poi_timeseries


In [176]:
poi_ts_created_by_complex = generate_poi_ts('CREATED', 'closest_complex_id', frequency)
poi_ts_created_by_complex = poi_ts_created_by_complex[(poi_ts_created_by_complex.index >= pd.to_datetime('2020-07-01')) & (poi_ts_created_by_complex.index <= pd.to_datetime('2024-12-31'))]
poi_ts_created_by_complex.to_csv(f'Data/Master Sets/NYC Master/NYC_Subway_POI_{frequency}_vts_created.csv')

In [177]:
poi_ts_modified_by_complex = generate_poi_ts('MODIFIED', 'closest_complex_id', frequency)
poi_ts_modified_by_complex = poi_ts_modified_by_complex[(poi_ts_modified_by_complex.index >= pd.to_datetime('2020-07-01')) & (poi_ts_modified_by_complex.index <= pd.to_datetime('2024-12-31'))]
poi_ts_modified_by_complex.to_csv(f'Data/Master Sets/NYC Master/NYC_Subway_POI_{frequency}_vts_modified.csv')

C:\Users\shygu\AppData\Local\Temp\ipykernel_10792\416471232.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  poi_df[date_index] = pd.to_datetime(poi_df[date_index]).dt.to_period(period).dt.to_timestamp()  # Ensure datetime format


In [178]:
poi_ts_created_by_line = generate_poi_ts('CREATED', 'closest_line_name', frequency)
poi_ts_created_by_line = poi_ts_created_by_line[(poi_ts_created_by_line.index >= pd.to_datetime('2020-07-01')) & (poi_ts_created_by_line.index <= pd.to_datetime('2024-12-31'))]
poi_ts_created_by_line.to_csv(f'Data/Master Sets/NYC Master/NYC_Subway_POI_{frequency}_vts_created_by_Line.csv')

In [179]:
poi_ts_modified_by_line = generate_poi_ts('MODIFIED', 'closest_line_name', frequency)
poi_ts_modified_by_line = poi_ts_modified_by_line[(poi_ts_modified_by_line.index >= pd.to_datetime('2020-07-01')) & (poi_ts_modified_by_line.index <= pd.to_datetime('2024-12-31'))]
poi_ts_modified_by_line.to_csv(f'Data/Master Sets/NYC Master/NYC_Subway_POI_{frequency}_vts_modified_by_Line.csv')

In [180]:
poi_ts_created_by_line

,4th Av,63rd St,6th Av - Culver,8th Av - Fulton St,Astoria,Broadway,Broadway - 7Av,Broadway - Brighton,Canarsie,Clark St,...,Nostrand,Pelham,Queens - Archer,Queens Blvd,Rockaway,Sea Beach,Sea Beach / West End / Culver / Brighton,Second Av,Staten Island,West End
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01,374.0,201.0,761.0,1525.0,454.0,92.0,1509.0,828.0,928.0,128.0,...,253.0,1295.0,202.0,1285.0,569.0,137.0,126.0,207.0,1569.0,232.0
2020-08-01,375.0,201.0,762.0,1530.0,455.0,92.0,1517.0,832.0,935.0,130.0,...,257.0,1301.0,207.0,1301.0,569.0,137.0,126.0,207.0,1570.0,234.0
2020-09-01,380.0,201.0,775.0,1544.0,455.0,92.0,1518.0,836.0,942.0,132.0,...,262.0,1305.0,209.0,1305.0,571.0,140.0,126.0,207.0,1573.0,236.0
2020-10-01,380.0,201.0,780.0,1552.0,469.0,92.0,1523.0,836.0,948.0,132.0,...,263.0,1315.0,209.0,1339.0,584.0,140.0,127.0,210.0,1576.0,236.0
2020-11-01,386.0,202.0,784.0,1561.0,474.0,95.0,1525.0,846.0,955.0,132.0,...,264.0,1315.0,212.0,1353.0,584.0,141.0,128.0,210.0,1576.0,236.0
2020-12-01,408.0,202.0,805.0,1585.0,474.0,95.0,1525.0,881.0,981.0,134.0,...,283.0,1315.0,212.0,1358.0,584.0,141.0,132.0,210.0,1578.0,258.0
2021-01-01,408.0,202.0,805.0,1585.0,474.0,95.0,1525.0,881.0,981.0,134.0,...,283.0,1315.0,212.0,1359.0,584.0,141.0,132.0,210.0,1579.0,258.0
2021-02-01,408.0,202.0,807.0,1585.0,474.0,95.0,1526.0,881.0,981.0,135.0,...,283.0,1315.0,212.0,1364.0,584.0,141.0,132.0,210.0,1579.0,258.0
2021-03-01,408.0,202.0,808.0,1585.0,474.0,95.0,1526.0,881.0,981.0,135.0,...,283.0,1315.0,212.0,1364.0,584.0,141.0,135.0,210.0,1579.0,258.0
